<a href="https://colab.research.google.com/github/Pataweepr/applyML_vistec_2019/blob/master/hw3_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import library 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
# from sklearn import datasets

# load data from kaggle 
train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
train = pd.read_csv(train_url) #training set
test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"
test = pd.read_csv(test_url) #test set


# เน้นการ vitualization data 
# หา mode หา mean ดู head tail 

In [0]:
################################################################################
# TODO#1:                                                                      #
# modify NA data from the 'age' column with mode or median in the dataframe  
# by using ".fillna" 
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

<details>
    <summary>SOLUTION HERE!</summary>
    <pre>
        <code>

test_generator = cnn_input_generators[2]
test_size = len(cnn_input_generators[2].filenames)
print("Total test image: ", test_size)
print("Metrics:", cnn_model.metrics_names)
cnn_model.evaluate_generator(test_generator, steps=test_size//32)
# It takes some times to evaluate the whole test set. You can evaluate on the sample of 3,200 data points by setting steps=100

        </code>
    </pre>
</details>

In [0]:
################################################################################
# TODO#2:                                                                      #
# convert data from 'Embarked' column
# represent S with 0, C with 1 and Q with 2 
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

In [0]:
################################################################################
# TODO#3:                                                                      #
# convert data from 'Sex' column
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

In [0]:
# exact feature form Pandas to Numpy 

data_train =  np.array(train[["PClass","Sex","Age","Embarked"]].values, dtype = float)
################################################################################
# TODO#4: using scikit learn creat Logistic regression model 
# and fit it with data_train
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

# logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')

In [0]:
################################################################################
# TODO#5:                                                                      #
# using the model with your test set
# Warning! Thresholding your result
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

In [0]:
################################################################################
# TODO#6:  Submit a screenshot of your submission (with the scores)
# Warning! check your format before submit to kaggle
# example data type
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

In [0]:
################################################################################
# optional task 1: RE-do with linear regression model 
# using scikit learn (gradirnt descent)
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

In [0]:
################################################################################
# optional task 2: solving linear regression model                                                                   #
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

In [0]:
################################################################################
# optional task 3: RE-do with linear regression model by using least square formular 
# and compare mse with solving by using scikit learn
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

In [0]:
################################################################################
# optional task 4: RE-do with linear regression model                                                                   #
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################